# Topics


In [88]:
import numpy as np
import pandas as pd
import math
import re
import sys
from shapely.geometry import Polygon
from matplotlib import pyplot as plt
from collections import Counter, OrderedDict, namedtuple, defaultdict, ChainMap
from queue import Queue
from copy import deepcopy
import networkx as nx
from functools import cmp_to_key, reduce
from itertools import product, permutations, combinations, combinations_with_replacement
from itertools import repeat
from functools import cache
from scipy.sparse import csr_matrix
from scipy.sparse.csgraph import maximum_flow
import json
import time
from z3 import *
from tqdm import tqdm

ModuleNotFoundError: No module named 'seaborn'

In [2]:
sys.setrecursionlimit(1500)

In [3]:
with open("17-input", "r") as file:
    lines = file.readlines()
data_raw = "".join(lines)
# data_raw = [line.replace("\n", "") for line in lines]
# data_raw = "\n".join(data_raw)
data_raw

'Register A: 59397658\nRegister B: 0\nRegister C: 0\n\nProgram: 2,4,1,1,7,5,4,6,1,4,0,3,5,5,3,0'

In [74]:
test_data_raw = r"""Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

test_data_raw2 = r"""Register A: 2024
Register B: 0
Register C: 0

Program: 0,3,5,4,3,0"""




def preprocess_data (data):
    registers, program = data.split("\n\n")
    registers = re.findall(r'-?\d+', registers)

    program = re.findall(r'-?\d+', program)
    return list(map(int, registers)), list(map(int,program)
)


test_data = preprocess_data(test_data_raw)
test_data2 = preprocess_data(test_data_raw2)
display(test_data)


([729, 0, 0], [0, 1, 5, 4, 3, 0])

In [6]:
full_data = preprocess_data(data_raw)
full_data

([59397658, 0, 0], [2, 4, 1, 1, 7, 5, 4, 6, 1, 4, 0, 3, 5, 5, 3, 0])

In [59]:

def solution (data, verbose=False):
    program = data[1]
    output = []
    registers = {"A": data[0][0], "B": data[0][1], "C": data[0][2]}
    opcode_desc = {0: "adv", 1:"bxl", 2: "bst", 3: "jnz", 4: "bxc", 5: "out", 6: "bdv", 7: "cdv"}

    Pointer = namedtuple("Pointer", ["pos", "jumped"])

    global pointer
    pointer = Pointer(0, False)

    def process_operand(operand, r):
        if 0 <= operand <= 3:
            return operand
        elif operand == 4:
            return r["A"] 
        elif operand == 5:
            return r["B"] 
        elif operand == 6:
            return r["C"] 
        elif operand == 7:
            raise ValueError("Operand 7 not allowed")

    def process_instruction(opcode, operand, r,  o: list):
        match opcode:
            case 0: # adv
                r["A"] = r["A"] // (2 ** process_operand(operand, r))
            case 1: # bxl
                r["B"] = r["B"] ^ operand
            case 2: # bst
                r["B"] = process_operand(operand, r) % 8
            case 3: # jnz
                if r["A"] != 0:
                    pointer = Pointer(operand, True)
            case 4: # bxc
                r["B"] = r["B"] ^ r["C"]
            case 5: # out
                o.append( process_operand(operand, r) % 8)
            case 6: # bdv
                r["B"] = r["A"] // (2 ** process_operand(operand, r))
            case 7: # cdv
                r["C"] = r["A"] // (2 ** process_operand(operand, r))

    
    

    while pointer.pos < len(program):
        if pointer.pos % 2 == 1:
            raise ValueError("Pointer points to operand")
        
        opcode = program[pointer.pos]
        operand = program[pointer.pos+1]

        # print(pointer.pos, opcode_desc[opcode], operand, "value:", process_operand(operand, registers))
        if opcode != 3:
            process_instruction(opcode, operand, registers, output)
        elif opcode == 3:
            if registers["A"] != 0:
                pointer = Pointer(operand, True)
        else:
            raise ValueError("Unknown opcode")

        # print(registers)
        if not pointer.jumped:
            pointer = Pointer(pointer.pos + 2, False)

        pointer = Pointer(pointer.pos, False)

    # print(registers)
    return ",".join(list(map(str, output)))


# 4,6,3,5,6,3,5,2,1,0
sol = solution(test_data, verbose=True)
print(sol)





4,6,3,5,6,3,5,2,1,0


In [60]:
# reg B = 1
example1 = r"""Register A: 729
Register B: 0
Register C: 9

Program: 2,6"""

example2 = r"""Register A: 10
Register B: 0
Register C: 0

Program: 5,0,5,1,5,4"""

# reg A  = 0
example3 = r"""Register A: 2024
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

# reg B = 26
example4 = r"""Register A: 2024
Register B: 29
Register C: 0

Program: 1,7"""

# reg B = 44354
example5 = r"""Register A: 2024
Register B: 2024
Register C: 43690

Program: 4,0"""


examples = {example1: "", example2: "0,1,2", example3: "4,2,5,6,7,7,7,7,3,1,0", example4: "", example5: ""}

def test_part1(examples):
    for idx, (ex, answer) in enumerate(examples.items()):
        data_prep = preprocess_data(ex)
        try:
            sol = solution(data_prep)
            if sol != answer:
                print(f"Test example {idx+1} failed.")
            else:
                print(f"Test {idx+1} successful.")
        except:
            print(f"Test {idx+1} invalid")
        
    


test_part1(examples)

Test 1 successful.
Test 2 successful.
Test 3 successful.
Test 4 successful.
Test 5 successful.


In [62]:
sol = solution(full_data)
print(sol)

4,6,1,4,2,1,3,1,6


# Part 2

In [162]:

def solution2 (data, verbose=False):

    def prep_output(o):
        return ",".join(list(map(str, o)))

    program = data[1]
    
    registers = {"A": data[0][0], "B": data[0][1], "C": data[0][2]}
    opcode_desc = {0: "adv", 1:"bxl", 2: "bst", 3: "jnz", 4: "bxc", 5: "out", 6: "bdv", 7: "cdv"}

    Pointer = namedtuple("Pointer", ["pos", "jumped"])
    
    def run_programm(registers, program):
        output = []

        global pointer
        pointer = Pointer(0, False)

        def process_operand(operand, r):
            if 0 <= operand <= 3:
                return operand
            elif operand == 4:
                return r["A"] 
            elif operand == 5:
                return r["B"] 
            elif operand == 6:
                return r["C"] 
            elif operand == 7:
                raise ValueError("Operand 7 not allowed")

        def process_instruction(opcode, operand, r,  o: list):
            match opcode:
                case 0: # adv
                    r["A"] = r["A"] // (2 ** process_operand(operand, r))
                case 1: # bxl
                    r["B"] = r["B"] ^ operand
                case 2: # bst
                    r["B"] = process_operand(operand, r) % 8
                case 3: # jnz
                    if r["A"] != 0:
                        pointer = Pointer(operand, True)
                case 4: # bxc
                    r["B"] = r["B"] ^ r["C"]
                case 5: # out
                    o.append( process_operand(operand, r) % 8)
                case 6: # bdv
                    r["B"] = r["A"] // (2 ** process_operand(operand, r))
                case 7: # cdv
                    r["C"] = r["A"] // (2 ** process_operand(operand, r))

        while pointer.pos < len(program):
            if pointer.pos % 2 == 1:
                raise ValueError("Pointer points to operand")
            
            opcode = program[pointer.pos]
            operand = program[pointer.pos+1]
            if opcode != 3:
                process_instruction(opcode, operand, registers, output)
            elif opcode == 3:
                if registers["A"] != 0:
                    pointer = Pointer(operand, True)
            else:
                raise ValueError("Unknown opcode")

            if not pointer.jumped:
                pointer = Pointer(pointer.pos + 2, False)

            pointer = Pointer(pointer.pos, False)
        return output


    previous_corrects = [0]
    for level, num in enumerate(reversed(program)):
        correct = []
        for start in previous_corrects:
            for i in range(8):
                val = start*8 + i
                registers =  {"A": val, "B": data[0][1], "C": data[0][2]}
                output = run_programm(registers, program)
                if output[0] == num:
                    correct.append(val)
        
        previous_corrects = correct

    return min(previous_corrects)


sol = solution2(test_data2, verbose=True)
print(sol)

117440


In [163]:
# [2, 4, 1, 1, 7, 5, 4, 6, 1, 4, 0, 3, 5, 5, 3, 0]
sol = solution2(full_data)
print(sol)

202366627359274
